# VCNet - 连续处理效应的深度学习方法

---

## 学习目标

完成本练习后，你将能够：

1. 理解连续处理 (Continuous Treatment) 与二元处理的本质区别
2. 掌握剂量-响应曲线 (Dose-Response Curve) 的概念
3. 理解 VCNet 的变系数网络设计
4. 实现 VCNet 并估计连续处理效应
5. 应用于真实业务场景：优惠券面额优化

---

## 真实场景：优惠券面额该发多少？

### 场景描述

你是某电商平台的增长负责人，手握 500 万优惠券预算。

**传统做法**：
- 要么发 10 元券，要么不发
- 用 Uplift Modeling 区分「发」vs「不发」

**新的问题**：
> **发 5 元还是 10 元还是 20 元？不同用户的「最优面额」是多少？**

这就是**连续处理效应 (Continuous Treatment Effect)** 问题！

### 与二元处理的区别

| 类型 | 二元处理 | 连续处理 |
|------|---------|----------|
| **处理变量** | T ∈ {0, 1} | T ∈ [0, ∞) 或 [t_min, t_max] |
| **目标** | 估计 Y(1) - Y(0) | 估计整个 Y(t) 曲线 |
| **业务问题** | 发不发券？ | 发多少面额？|
| **倾向得分** | P(T=1|X) | f(T|X) 概率密度 |
| **因果识别** | 更简单 | 需要更强假设 |

### 生动比喻：调音量

想象你在调音响的音量：

- **二元处理**：音量只有「开」和「关」两档
- **连续处理**：音量可以从 0 调到 100 任意值

问题是：**每个人感觉舒适的音量不同，如何为每个人找到最优音量？**

---

## 核心概念：剂量-响应曲线

### 剂量-响应曲线 (Dose-Response Curve)

对于个体 $i$，我们想估计：

$$\mu(t, x) = E[Y | T = t, X = x]$$

这是一条**曲线**，描述了处理强度 $t$ 与结果 $Y$ 的关系。

```
    Y (结果)
    │
    │         ╭───────╮
    │       ╭╯         ╲
    │     ╭╯            ╲
    │   ╭╯               ╲_____
    │ ╭╯
    │╭╯
    └─────────────────────────── T (处理强度/剂量)
         ↑       ↑         ↑
      起效点   最优点    边际递减
```

### 连续处理效应

**平均剂量-响应函数 (ADRF)**：
$$\mu(t) = E[Y(t)] = E_X[E[Y|T=t, X]]$$

**边际处理效应 (Marginal Treatment Effect)**：
$$\frac{\partial \mu(t, x)}{\partial t}$$

**最优处理强度**：
$$t^*(x) = \arg\max_t \mu(t, x)$$

---

## 连续处理的因果识别

### 广义倾向得分 (Generalized Propensity Score)

对于连续处理，倾向得分变成**概率密度函数**：

$$e(t|x) = f_{T|X}(t|x)$$

### 识别假设

1. **弱无混淆 (Weak Unconfoundedness)**：
   $$Y(t) \perp T | X, \quad \forall t$$

2. **共同支撑 (Common Support)**：
   $$0 < e(t|x) < \infty, \quad \forall t, x$$

3. **平滑性 (Smoothness)**：
   $$\mu(t, x) \text{ 对 } t \text{ 足够光滑}$$

---

## VCNet: Varying Coefficient Network

### 核心创新：变系数设计

VCNet 的核心思想是：**让神经网络的权重随处理强度变化**。

传统神经网络：
$$y = W \cdot \phi(x)$$

VCNet：
$$y = W(t) \cdot \phi(x)$$

其中 $W(t)$ 是**处理强度 $t$ 的函数**！

### 架构图解

```
┌─────────────────────────────────────────────────────────────┐
│                        VCNet 架构                           │
├─────────────────────────────────────────────────────────────┤
│                                                             │
│   输入 X ───→ [表示网络 Φ] ───→ φ(X)                        │
│               (与 TARNet 类似)       ╲                      │
│                                        ╲                    │
│   输入 T ───→ [系数网络 ψ] ───→ W(T) ───→ 点乘 → Ŷ(T)       │
│               (Spline 基函数)         ╱                     │
│                                      ╱                      │
│                                                             │
│   关键: W(T) · φ(X) 实现了「处理-特征」的交互建模            │
│                                                             │
└─────────────────────────────────────────────────────────────┘
```

### 为什么用样条基函数？

为了让 $W(t)$ 对 $t$ 足够光滑，VCNet 使用**样条基函数 (Spline Basis)**：

$$W(t) = \sum_{k=1}^{K} \alpha_k \cdot B_k(t)$$

其中 $B_k(t)$ 是第 $k$ 个样条基函数。

这保证了剂量-响应曲线是**光滑**的，避免了 overfitting。

---

In [ ]:
# 环境准备
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
from scipy.interpolate import BSpline
from typing import Tuple, Dict, List
import warnings
warnings.filterwarnings('ignore')

# 设置随机种子
np.random.seed(42)
torch.manual_seed(42)

# 颜色方案
COLORS = {
    'primary': '#2D9CDB',
    'success': '#27AE60',
    'danger': '#EB5757',
    'warning': '#F2994A',
    'info': '#9B51E0',
}

print(f"PyTorch 版本: {torch.__version__}")
print("环境准备完成！")

---

## Part 1: 数据生成

### 场景：优惠券面额优化

In [ ]:
def generate_coupon_data(n_samples: int = 5000, seed: int = 42) -> Tuple:
    """
    生成优惠券面额优化数据
    
    场景：
    - X: 用户特征（购买力、活跃度、历史消费等）
    - T: 优惠券面额（0-50 元）
    - Y: 用户消费金额
    
    真实关系：
    - 不同购买力的用户，最优面额不同
    - 高购买力用户：边际递减快
    - 低购买力用户：需要更大面额才能激活
    """
    np.random.seed(seed)
    
    # 用户特征 (5 维)
    X = np.random.randn(n_samples, 5)
    # X[:, 0]: 购买力（标准化）
    # X[:, 1]: 活跃度
    # X[:, 2]: 历史消费金额
    # X[:, 3]: 价格敏感度
    # X[:, 4]: 品类偏好
    
    # 模拟真实的优惠券分配（非随机）
    # 活跃用户更可能收到优惠券，且面额更高
    propensity_mean = 15 + 5 * X[:, 1] + 3 * X[:, 3]  # 倾向性影响面额
    propensity_mean = np.clip(propensity_mean, 0, 50)
    T = np.clip(propensity_mean + np.random.randn(n_samples) * 8, 0, 50)
    
    # 归一化到 [0, 1] 用于模型
    T_normalized = T / 50.0
    
    # 真实的剂量-响应关系（复杂的非线性）
    def true_dose_response(t, x):
        """
        真实的剂量-响应函数
        
        - 基础消费取决于用户特征
        - 优惠券效应有边际递减
        - 不同用户的最优面额不同
        """
        # 基础消费
        base = 100 + 20 * x[:, 0] + 15 * x[:, 2]
        
        # 优惠券效应（对数函数 = 边际递减）
        # 高价格敏感度用户对优惠券响应更大
        sensitivity = 1 + 0.5 * x[:, 3]
        coupon_effect = sensitivity * 30 * np.log1p(t * 5)
        
        # 低购买力用户需要更大面额才能激活（门槛效应）
        threshold = np.maximum(0, -x[:, 0] * 0.3)
        activation = 1 / (1 + np.exp(-10 * (t - threshold)))
        
        return base + coupon_effect * activation
    
    # 生成结果
    noise = np.random.randn(n_samples) * 10
    Y = true_dose_response(T_normalized, X) + noise
    
    return X, T, T_normalized, Y, true_dose_response


# 生成数据
X, T, T_norm, Y, true_dr = generate_coupon_data(n_samples=5000)

print("数据生成完成！")
print(f"样本数: {len(X)}")
print(f"特征维度: {X.shape[1]}")
print(f"\n优惠券面额统计:")
print(f"  范围: [{T.min():.1f}, {T.max():.1f}] 元")
print(f"  均值: {T.mean():.1f} 元")
print(f"  标准差: {T.std():.1f} 元")
print(f"\n消费金额统计:")
print(f"  范围: [{Y.min():.1f}, {Y.max():.1f}] 元")
print(f"  均值: {Y.mean():.1f} 元")

In [ ]:
# 可视化真实的剂量-响应关系
fig = make_subplots(rows=1, cols=2,
                    subplot_titles=('优惠券面额分布', '不同用户的剂量-响应曲线'))

# 面额分布
fig.add_trace(go.Histogram(x=T, nbinsx=30, name='面额分布',
                           marker_color=COLORS['primary']), row=1, col=1)

# 剂量-响应曲线
t_range = np.linspace(0, 1, 50)

# 不同用户类型
user_types = [
    ('高购买力', np.array([[1.5, 0, 1, 0, 0]])),
    ('中购买力', np.array([[0, 0, 0, 0, 0]])),
    ('低购买力', np.array([[-1.5, 0, -1, 0.5, 0]])),
]

colors = [COLORS['success'], COLORS['warning'], COLORS['danger']]

for (label, x), color in zip(user_types, colors):
    x_repeat = np.repeat(x, len(t_range), axis=0)
    y_curve = true_dr(t_range, x_repeat)
    fig.add_trace(go.Scatter(x=t_range * 50, y=y_curve, name=label,
                             line=dict(color=color, width=2)), row=1, col=2)

fig.update_xaxes(title_text='优惠券面额 (元)', row=1, col=1)
fig.update_xaxes(title_text='优惠券面额 (元)', row=1, col=2)
fig.update_yaxes(title_text='用户数', row=1, col=1)
fig.update_yaxes(title_text='预期消费 (元)', row=1, col=2)

fig.update_layout(height=400, template='plotly_white',
                  title_text='优惠券数据探索')
fig.show()

print("""
📊 观察:
1. 优惠券面额呈正态分布，集中在 10-20 元
2. 不同用户的剂量-响应曲线形状不同
3. 低购买力用户需要更大面额才能激活（有门槛）
4. 所有曲线都呈现边际递减
""")

---

## Part 2: VCNet 实现

### 2.1 样条基函数

In [ ]:
class TruncatedBasis(nn.Module):
    """
    截断幂基函数 (Truncated Power Basis)
    
    用于生成样条基函数，使得 W(t) 对 t 光滑
    
    B_k(t) = max(0, t - ξ_k)^p
    
    其中 ξ_k 是节点 (knots)，p 是多项式阶数
    """
    
    def __init__(self, num_knots: int = 5, degree: int = 2):
        super().__init__()
        
        self.num_knots = num_knots
        self.degree = degree
        
        # 均匀分布的节点
        knots = torch.linspace(0, 1, num_knots + 2)[1:-1]  # 排除边界
        self.register_buffer('knots', knots)
    
    def forward(self, t: torch.Tensor) -> torch.Tensor:
        """
        计算基函数值
        
        Args:
            t: 处理强度，形状 (batch_size, 1)
        
        Returns:
            basis: 基函数值，形状 (batch_size, num_knots + degree + 1)
        """
        if len(t.shape) == 1:
            t = t.unsqueeze(1)
        
        batch_size = t.shape[0]
        
        # 多项式项: 1, t, t^2, ..., t^p
        poly_terms = [torch.ones(batch_size, 1, device=t.device)]
        for d in range(1, self.degree + 1):
            poly_terms.append(t ** d)
        
        # 截断幂项: max(0, t - ξ_k)^p
        truncated_terms = []
        for k in range(self.num_knots):
            diff = t - self.knots[k]
            truncated = torch.relu(diff) ** self.degree
            truncated_terms.append(truncated)
        
        # 拼接所有基函数
        basis = torch.cat(poly_terms + truncated_terms, dim=1)
        
        return basis


# 测试基函数
basis_fn = TruncatedBasis(num_knots=5, degree=2)
t_test = torch.linspace(0, 1, 100).unsqueeze(1)
basis_values = basis_fn(t_test)

print(f"基函数数量: {basis_values.shape[1]}")
print(f"输入形状: {t_test.shape}")
print(f"输出形状: {basis_values.shape}")

In [ ]:
# 可视化基函数
fig = go.Figure()

t_plot = t_test.numpy().squeeze()
basis_plot = basis_values.detach().numpy()

for i in range(basis_plot.shape[1]):
    fig.add_trace(go.Scatter(x=t_plot, y=basis_plot[:, i],
                             name=f'B_{i}', mode='lines'))

fig.update_layout(
    title='截断幂基函数',
    xaxis_title='处理强度 t',
    yaxis_title='基函数值',
    template='plotly_white',
    height=400
)
fig.show()

print("""
📊 基函数解释:
- B_0 = 1 (常数项)
- B_1 = t (线性项)
- B_2 = t^2 (二次项)
- B_3+ = max(0, t - ξ_k)^2 (截断幂项，在节点处开始非零)
""")

### 2.2 VCNet 模型

In [ ]:
class VCNet(nn.Module):
    """
    Varying Coefficient Network (VCNet)
    
    核心思想:
    - 表示网络: 学习用户特征的表示 φ(X)
    - 系数网络: 让权重随处理强度变化 W(t)
    - 输出: Y = W(t) · φ(X)
    """
    
    def __init__(
        self,
        input_dim: int,
        hidden_dim: int = 64,
        repr_dim: int = 32,
        num_knots: int = 5,
        spline_degree: int = 2
    ):
        super().__init__()
        
        self.repr_dim = repr_dim
        
        # 表示网络 Φ(X)
        self.representation = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ELU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ELU(),
            nn.Linear(hidden_dim, repr_dim),
            nn.ELU(),
        )
        
        # 样条基函数
        self.basis = TruncatedBasis(num_knots=num_knots, degree=spline_degree)
        self.num_basis = num_knots + spline_degree + 1
        
        # 系数网络：从基函数值到变系数 W(t)
        # W(t) 的形状是 (repr_dim,)，所以我们需要 repr_dim 组系数
        self.coef_network = nn.Sequential(
            nn.Linear(self.num_basis, hidden_dim),
            nn.ELU(),
            nn.Linear(hidden_dim, repr_dim),
        )
        
        # 偏置项（也随 t 变化）
        self.bias_network = nn.Sequential(
            nn.Linear(self.num_basis, hidden_dim // 2),
            nn.ELU(),
            nn.Linear(hidden_dim // 2, 1),
        )
    
    def forward(self, x: torch.Tensor, t: torch.Tensor) -> torch.Tensor:
        """
        前向传播
        
        Args:
            x: 用户特征，形状 (batch_size, input_dim)
            t: 处理强度，形状 (batch_size,) 或 (batch_size, 1)
        
        Returns:
            y: 预测结果，形状 (batch_size, 1)
        """
        # 用户表示
        phi = self.representation(x)  # (batch, repr_dim)
        
        # 基函数值
        if len(t.shape) == 1:
            t = t.unsqueeze(1)
        basis = self.basis(t)  # (batch, num_basis)
        
        # 变系数 W(t)
        W_t = self.coef_network(basis)  # (batch, repr_dim)
        
        # 偏置 b(t)
        b_t = self.bias_network(basis)  # (batch, 1)
        
        # 输出: Y = W(t) · φ(X) + b(t)
        y = torch.sum(W_t * phi, dim=1, keepdim=True) + b_t
        
        return y
    
    def predict_dose_response(self, x: torch.Tensor, t_values: torch.Tensor) -> torch.Tensor:
        """
        预测完整的剂量-响应曲线
        
        Args:
            x: 单个用户特征，形状 (1, input_dim)
            t_values: 处理强度值列表，形状 (n_points,)
        
        Returns:
            y_values: 对应的预测结果，形状 (n_points,)
        """
        n_points = len(t_values)
        x_repeat = x.repeat(n_points, 1)
        y_values = self.forward(x_repeat, t_values)
        return y_values.squeeze()


# 测试模型
vcnet = VCNet(input_dim=5, hidden_dim=64, repr_dim=32)
X_test = torch.FloatTensor(X[:10])
T_test = torch.FloatTensor(T_norm[:10])

y_pred = vcnet(X_test, T_test)
print(f"VCNet 测试通过！")
print(f"输入: X={X_test.shape}, T={T_test.shape}")
print(f"输出: Y={y_pred.shape}")

---

## Part 3: 训练 VCNet

In [ ]:
def train_vcnet(
    model: VCNet,
    X: np.ndarray,
    T: np.ndarray,
    Y: np.ndarray,
    n_epochs: int = 300,
    batch_size: int = 128,
    learning_rate: float = 1e-3,
    weight_decay: float = 1e-4,
    verbose: bool = True
) -> Dict:
    """
    训练 VCNet
    """
    # 数据准备
    X_tensor = torch.FloatTensor(X)
    T_tensor = torch.FloatTensor(T)
    Y_tensor = torch.FloatTensor(Y).unsqueeze(1)
    
    dataset = TensorDataset(X_tensor, T_tensor, Y_tensor)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
    
    # 优化器
    optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=100, gamma=0.5)
    
    # 损失函数
    mse_loss = nn.MSELoss()
    
    # 训练历史
    history = {'loss': []}
    
    for epoch in range(n_epochs):
        epoch_loss = 0.0
        n_batches = 0
        
        for batch_x, batch_t, batch_y in dataloader:
            optimizer.zero_grad()
            
            # 前向传播
            y_pred = model(batch_x, batch_t)
            
            # 损失
            loss = mse_loss(y_pred, batch_y)
            
            # 反向传播
            loss.backward()
            optimizer.step()
            
            epoch_loss += loss.item()
            n_batches += 1
        
        scheduler.step()
        
        avg_loss = epoch_loss / n_batches
        history['loss'].append(avg_loss)
        
        if verbose and (epoch + 1) % 50 == 0:
            print(f"Epoch {epoch+1}/{n_epochs}, Loss: {avg_loss:.4f}")
    
    return history


# 训练模型
print("开始训练 VCNet...")
print("="*60)

vcnet_model = VCNet(input_dim=X.shape[1], hidden_dim=64, repr_dim=32, num_knots=5)
history = train_vcnet(
    vcnet_model, X, T_norm, Y,
    n_epochs=300,
    batch_size=128,
    learning_rate=1e-3,
    verbose=True
)

print("="*60)
print("训练完成！")

---

## Part 4: 评估与可视化

In [ ]:
# 评估模型
def evaluate_vcnet(model: VCNet, X: np.ndarray, T: np.ndarray, Y: np.ndarray,
                   true_dr_func) -> Dict:
    """
    评估 VCNet 性能
    """
    model.eval()
    with torch.no_grad():
        X_tensor = torch.FloatTensor(X)
        T_tensor = torch.FloatTensor(T)
        Y_pred = model(X_tensor, T_tensor).numpy().squeeze()
    
    # MSE
    mse = np.mean((Y - Y_pred) ** 2)
    rmse = np.sqrt(mse)
    
    # 评估剂量-响应曲线的准确性
    # 在多个 t 值上比较
    t_eval = np.linspace(0, 1, 20)
    dr_errors = []
    
    for i in range(min(100, len(X))):  # 抽样评估
        x = X[i:i+1]
        x_repeat = np.repeat(x, len(t_eval), axis=0)
        
        # 真实曲线
        true_curve = true_dr_func(t_eval, x_repeat)
        
        # 预测曲线
        with torch.no_grad():
            pred_curve = model.predict_dose_response(
                torch.FloatTensor(x), 
                torch.FloatTensor(t_eval)
            ).numpy()
        
        dr_errors.append(np.mean((true_curve - pred_curve) ** 2))
    
    dr_mse = np.mean(dr_errors)
    
    return {
        'rmse': rmse,
        'dr_mse': dr_mse,
        'Y_pred': Y_pred
    }


metrics = evaluate_vcnet(vcnet_model, X, T_norm, Y, true_dr)

print("VCNet 评估结果")
print("="*60)
print(f"预测 RMSE: {metrics['rmse']:.4f}")
print(f"剂量-响应曲线 MSE: {metrics['dr_mse']:.4f}")

In [ ]:
# 可视化剂量-响应曲线
vcnet_model.eval()

fig = make_subplots(rows=1, cols=3,
                    subplot_titles=('高购买力用户', '中购买力用户', '低购买力用户'))

t_range = np.linspace(0, 1, 50)

user_types = [
    ('高购买力', np.array([[1.5, 0, 1, 0, 0]])),
    ('中购买力', np.array([[0, 0, 0, 0, 0]])),
    ('低购买力', np.array([[-1.5, 0, -1, 0.5, 0]])),
]

for col, (label, x) in enumerate(user_types, 1):
    x_repeat = np.repeat(x, len(t_range), axis=0)
    
    # 真实曲线
    true_curve = true_dr(t_range, x_repeat)
    
    # 预测曲线
    with torch.no_grad():
        pred_curve = vcnet_model.predict_dose_response(
            torch.FloatTensor(x),
            torch.FloatTensor(t_range)
        ).numpy()
    
    fig.add_trace(go.Scatter(x=t_range * 50, y=true_curve, name='真实',
                             line=dict(color=COLORS['primary'], width=2),
                             showlegend=(col==1)), row=1, col=col)
    fig.add_trace(go.Scatter(x=t_range * 50, y=pred_curve, name='VCNet预测',
                             line=dict(color=COLORS['danger'], width=2, dash='dash'),
                             showlegend=(col==1)), row=1, col=col)

fig.update_xaxes(title_text='优惠券面额 (元)')
fig.update_yaxes(title_text='预期消费 (元)', row=1, col=1)

fig.update_layout(height=400, template='plotly_white',
                  title_text='VCNet 剂量-响应曲线预测 vs 真实')
fig.show()

print("""
📊 观察:
1. VCNet 能够捕捉不同用户的剂量-响应曲线形状
2. 边际递减效应被正确学习
3. 低购买力用户的门槛效应也被捕捉到
""")

---

## Part 5: 业务应用 - 最优面额推荐

In [ ]:
def find_optimal_dosage(model: VCNet, x: np.ndarray, 
                         cost_per_unit: float = 1.0,
                         t_range: Tuple[float, float] = (0, 1),
                         n_points: int = 100) -> Dict:
    """
    寻找用户的最优处理强度
    
    考虑 ROI：最大化 (响应 - 成本)
    
    Args:
        cost_per_unit: 每单位处理的成本
    
    Returns:
        最优处理强度和对应的预测响应
    """
    model.eval()
    
    t_values = torch.linspace(t_range[0], t_range[1], n_points)
    
    with torch.no_grad():
        x_tensor = torch.FloatTensor(x)
        y_pred = model.predict_dose_response(x_tensor, t_values).numpy()
    
    # 计算净收益 = 响应 - 成本
    costs = t_values.numpy() * cost_per_unit * 50  # 转回实际面额
    net_benefit = y_pred - costs
    
    # 找到最优点
    optimal_idx = np.argmax(net_benefit)
    optimal_t = t_values[optimal_idx].item()
    optimal_y = y_pred[optimal_idx]
    optimal_cost = costs[optimal_idx]
    optimal_net = net_benefit[optimal_idx]
    
    return {
        'optimal_t': optimal_t,
        'optimal_amount': optimal_t * 50,
        'expected_spending': optimal_y,
        'cost': optimal_cost,
        'net_benefit': optimal_net,
        'roi': (optimal_y - optimal_cost) / optimal_cost if optimal_cost > 0 else float('inf'),
        't_values': t_values.numpy(),
        'y_values': y_pred,
        'net_benefits': net_benefit
    }


# 为不同用户找最优面额
print("为不同用户寻找最优优惠券面额...")
print("="*60)

for label, x in user_types:
    result = find_optimal_dosage(vcnet_model, x, cost_per_unit=0.5)
    
    print(f"\n{label}:")
    print(f"  最优面额: {result['optimal_amount']:.1f} 元")
    print(f"  预期消费: {result['expected_spending']:.1f} 元")
    print(f"  优惠券成本: {result['cost']:.1f} 元")
    print(f"  净收益: {result['net_benefit']:.1f} 元")
    print(f"  ROI: {result['roi']:.2f}")

In [ ]:
# 可视化最优面额分析
fig = make_subplots(rows=1, cols=3,
                    subplot_titles=('高购买力', '中购买力', '低购买力'))

for col, (label, x) in enumerate(user_types, 1):
    result = find_optimal_dosage(vcnet_model, x, cost_per_unit=0.5)
    
    t_plot = result['t_values'] * 50
    
    # 预期消费
    fig.add_trace(go.Scatter(x=t_plot, y=result['y_values'], name='预期消费',
                             line=dict(color=COLORS['primary']),
                             showlegend=(col==1)), row=1, col=col)
    
    # 净收益
    fig.add_trace(go.Scatter(x=t_plot, y=result['net_benefits'], name='净收益',
                             line=dict(color=COLORS['success']),
                             showlegend=(col==1)), row=1, col=col)
    
    # 最优点
    fig.add_trace(go.Scatter(x=[result['optimal_amount']], y=[result['net_benefit']],
                             mode='markers', marker=dict(size=12, color=COLORS['danger']),
                             name='最优点', showlegend=(col==1)), row=1, col=col)

fig.update_xaxes(title_text='优惠券面额 (元)')
fig.update_yaxes(title_text='金额 (元)', row=1, col=1)

fig.update_layout(height=400, template='plotly_white',
                  title_text='个性化最优优惠券面额')
fig.show()

---

## 思考题

### 基础理解

1. **为什么 VCNet 用变系数设计而不是简单地把 T 作为特征输入？** 这有什么优势？

2. **样条基函数的作用是什么？** 如果不用样条，直接让 W(t) = NN(t)，会有什么问题？

### 深入分析

3. **连续处理的共同支撑假设比二元处理更难满足吗？** 为什么？

4. **如何处理处理强度分布不均匀的问题？** 例如，大部分用户收到 10 元券，很少用户收到 40 元券。

### 实战应用

5. **在优惠券场景中，如何设计 A/B 测试来收集训练数据？** 需要注意什么？

6. **如果有预算约束，如何在整体上优化优惠券分配？** 这与 Part 6 的预算分配优化有什么联系？

---

## 思考题参考答案

### 1. 为什么 VCNet 用变系数设计而不是简单地把 T 作为特征输入？

**参考答案**:

变系数设计 vs 简单拼接特征的本质区别在于**交互建模的方式**。

**方法对比**:

| 方法 | 架构 | 优势 | 劣势 |
|------|------|------|------|
| **简单拼接** | Y = NN([X, T]) | 简单易实现 | 难以学习复杂交互 |
| **VCNet** | Y = W(T) · φ(X) | 显式建模T-X交互 | 架构更复杂 |

**为什么简单拼接不够？**

```python
# 方法1: 简单拼接（不推荐）
class NaiveNet(nn.Module):
    def forward(self, x, t):
        # 直接拼接
        concat = torch.cat([x, t.unsqueeze(1)], dim=1)
        return self.network(concat)
```

**问题**:
1. **有限的交互能力**: MLP虽然理论上可以逼近任何函数，但对于T和X的交互，需要非常深的网络
2. **无光滑性保证**: 对于相近的t值，预测可能剧烈波动
3. **样本效率低**: 每个t值的数据是独立学习的，无法利用相邻t值的信息

**VCNet的优势**:

```python
# 方法2: VCNet
class VCNet(nn.Module):
    def forward(self, x, t):
        phi = self.representation(x)      # 学习特征表示
        W_t = self.coef_network(t)        # t的系数函数
        return torch.sum(W_t * phi, dim=1) + self.bias(t)
```

**优势**:

1. **显式交互建模**:
   - Y = W(t) · φ(x) 直接建模"处理强度如何调制特征的影响"
   - 这是multiplicative interaction，比加性更强大

2. **样条基函数的正则化**:
   - W(t) = Σ α_k · B_k(t) 保证对t的光滑性
   - 相近的t值自然有相近的权重
   - 防止overfitting

3. **可解释性**:
   - W(t)的变化直接反映"处理强度如何改变特征重要性"
   - 可以可视化W_k(t)曲线理解模型

**数学直觉**:

想象优惠券场景:
- 用户特征 φ(X) = [购买力, 活跃度, 价格敏感度, ...]
- 对于低面额券(t=0.1): 可能"价格敏感度"权重大 → W(0.1) = [0.1, 0.2, 0.8, ...]
- 对于高面额券(t=0.8): 可能"购买力"权重大 → W(0.8) = [0.7, 0.3, 0.2, ...]

VCNet通过W(t)自动学习这种"不同处理强度下特征重要性的变化"!

**实验证据**:

原论文实验显示VCNet在IHDP数据集上比简单拼接方法PEHE降低15-30%。

**结论**: 变系数设计是VCNet的核心创新，它通过W(t) · φ(x)显式建模处理-特征交互，并通过样条基函数保证光滑性，这是简单拼接无法做到的。

---

### 2. 样条基函数的作用是什么？如果不用样条，直接让 W(t) = NN(t)，会有什么问题？

**参考答案**:

样条基函数的作用是**强制W(t)对t光滑，防止overfitting**。

**样条 vs 神经网络对比**:

| 方法 | W(t)的形式 | 光滑性 | 可控性 | 样本效率 |
|------|-----------|--------|--------|---------|
| **样条基函数** | Σ α_k · B_k(t) | ✅ 强制光滑 | ✅ 可调节(节点数) | ✅ 高 |
| **MLP** | NN(t) | ❌ 无保证 | ❌ 黑盒 | ⚠️ 中等 |

**样条基函数的数学形式**:

$$W(t) = \sum_{k=1}^{K} \alpha_k \cdot B_k(t)$$

其中 $B_k(t)$ 是截断幂基函数:
- $B_1(t) = 1$ (常数项)
- $B_2(t) = t$ (线性项)
- $B_3(t) = t^2$ (二次项)
- $B_{3+k}(t) = \max(0, t - \xi_k)^2$ (截断幂项，在节点$\xi_k$处激活)

**为什么需要光滑性？**

1. **物理合理性**: 
   - 剂量-响应曲线在真实世界中通常是光滑的
   - 例如药物剂量从10mg到11mg，效果不会突变

2. **样本效率**:
   - 光滑性=归纳偏置
   - 在t=0.5处的观测可以帮助预测t=0.51的效果
   - 减少所需样本量

3. **泛化能力**:
   - 防止在数据稀疏的t区域overfitting
   - 通过邻近节点插值

**如果用NN(t)会怎样？**

```python
# 不推荐的做法
class BadVCNet(nn.Module):
    def __init__(self, ...):
        # 直接用MLP建模W(t)
        self.coef_network = nn.Sequential(
            nn.Linear(1, 64),  # 输入是标量t
            nn.ReLU(),
            nn.Linear(64, repr_dim)
        )
```

**问题**:

1. **过拟合风险**:
   - MLP可以拟合任意函数，包括高频振荡
   - 在数据稀疏区域可能产生不合理的W(t)

2. **训练不稳定**:
   - t是标量输入，信息量有限
   - MLP可能陷入局部最优

3. **无先验知识**:
   - MLP从零学习，忽略了"剂量响应应该光滑"这一先验

**可视化对比**:

```
样条基函数的W(t):
      │     ╭──────╮
W(t)  │   ╭╯        ╲╮
      │ ╭╯            ╲___
      └─────────────────────── t
         光滑、可控

MLP的W(t):
      │  ╱╲╱╲   ╱╲
W(t)  │ ╱    ╲╱╲╱  ╲╱╲
      │╱              ╲
      └─────────────────────── t
         可能过拟合、抖动
```

**超参数控制**:

样条基函数的光滑性可通过调节：
- **节点数K**: K越大，曲线越灵活
- **多项式阶数p**: p越高，连续性越强
- 这比调MLP的层数/宽度更直观

**最佳实践**:

1. **默认选择**: 5-10个节点，2次多项式（原论文推荐）
2. **数据量大**: 可以增加节点数
3. **先验强**: 减少节点数（如已知单调性，可用3-5个节点）

**结论**: 样条基函数通过有限的基函数组合保证W(t)的光滑性，这是一种强大的正则化，比直接用MLP更适合连续处理场景。

---

### 3. 连续处理的共同支撑假设比二元处理更难满足吗？为什么？

**参考答案**:

**是的，连续处理的共同支撑假设更难满足**，原因是"连续"带来的维度诅咒。

**共同支撑假设对比**:

| 处理类型 | 共同支撑要求 | 难度 |
|---------|-------------|------|
| **二元处理** | 0 < e(X) < 1, ∀X | 中等 |
| **连续处理** | 0 < f(t\\|X) < ∞, ∀t,X | 困难 |

**数学形式**:

**二元处理**:
$$0 < P(T=1 | X=x) < 1, \quad \forall x$$

只需要每个X既能被处理也能不被处理。

**连续处理**:
$$0 < f_{T|X}(t | X=x) < \infty, \quad \forall t \in \mathcal{T}, x$$

需要**每个X在所有可能的t值上都有观测**！

**为什么更难？**

#### 原因1: 维度爆炸

**二元**: 只需验证两个条件
- P(T=0|X) > 0
- P(T=1|X) > 0

**连续**: 需验证无穷多个条件
- f(t=0|X) > 0
- f(t=0.01|X) > 0
- f(t=0.02|X) > 0
- ...
- f(t=1|X) > 0

**实践中的挑战**: 即使T的取值范围是[0,1]，也很难保证每个X都在整个区间有观测。

#### 原因2: 数据稀疏性

想象优惠券场景:

```
用户1 (高购买力):
  观测到的券: 20元、25元、30元
  缺失区域: 0-15元、35-50元

用户2 (低购买力):
  观测到的券: 5元、10元、15元  
  缺失区域: 20-50元
```

对于用户1，我们能估计她在30元券的效果，但**无法**可靠估计5元券的效果（共同支撑违反！）

#### 原因3: 非随机分配

现实中处理强度往往不是随机的:
- 活跃用户→高面额券
- 新用户→低面额券

导致**条件分布的支撑不重叠**:

```
f(T|X=高购买力) 集中在 [20,50]
f(T|X=低购买力) 集中在 [0,20]
           ↓
在 [0,20] 区域，高购买力用户数据稀疏
在 [20,50] 区域，低购买力用户数据稀疏
```

**检验方法**:

```python
def check_common_support(X, T, n_bins=10):
    """
    检验连续处理的共同支撑
    """
    from sklearn.cluster import KMeans
    
    # 将X聚类成几个组
    kmeans = KMeans(n_clusters=5).fit(X)
    labels = kmeans.labels_
    
    # 将T分桶
    t_bins = np.linspace(T.min(), T.max(), n_bins+1)
    
    # 对每个X组和T桶，检查样本数
    for group in range(5):
        for i in range(n_bins):
            mask = (labels == group) & (T >= t_bins[i]) & (T < t_bins[i+1])
            n_samples = mask.sum()
            
            if n_samples < 5:  # 阈值
                print(f"警告: 组{group}在区间[{t_bins[i]:.1f}, {t_bins[i+1]:.1f}]样本不足({n_samples})")
```

**缓解策略**:

1. **增强数据收集**:
   - 设计**分层随机化**实验
   - 确保每个X层在所有t区间都有足够样本

2. **限制推断范围**:
   - 只在有充分支撑的t区间做推断
   - 例如只估计 t ∈ [0.2, 0.8] 的效应

3. **外推 with caution**:
   - 使用样条的光滑性外推到数据稀疏区域
   - 但要报告不确定性（置信区间会很宽）

4. **IPW重加权**:
   - 广义倾向得分 GPS = f(T|X)
   - 对稀疏区域上采样

**实际建议**:

在优惠券场景:
- 收集数据时: 确保各种用户都尝试各种面额（至少在几个关键档位）
- 建模时: 可以考虑离散化T（如5元、10元、20元、50元），降低维度
- 推断时: 对于数据稀疏的(X, T)组合，报告"不确定性高，建议做实验"

**结论**: 连续处理的共同支撑假设因维度爆炸和数据稀疏性更难满足，需要更仔细的实验设计和谨慎的外推。

---

### 4. 如何处理处理强度分布不均匀的问题？

**参考答案**:

处理强度分布不均匀是连续处理的常见问题，有多种策略：

**问题示例**:

```
优惠券数据分布:
  [0-10元):   ████████████████████ (40%)
  [10-20元):  ████████████████████████ (50%)
  [20-30元):  ████ (8%)
  [30-50元):  █ (2%)
```

大部分数据集中在10-20元，高面额券数据稀疏。

**策略1: 广义倾向得分加权 (GPS Weighting)**

**原理**: 用 1/f(T|X) 加权，提升稀疏区域样本的权重

```python
def train_with_gps_weighting(model, X, T, Y):
    # 估计 GPS: f(T|X)
    gps_model = GaussianProcessRegressor()  # 或其他密度估计
    gps_model.fit(X, T)
    
    # 计算每个样本的GPS
    gps = gps_model.pdf(T, X)  # 概率密度
    
    # 加权损失
    weights = 1.0 / (gps + 1e-6)
    weights = weights / weights.sum() * len(weights)  # 归一化
    
    loss = weighted_mse_loss(Y_pred, Y, weights)
```

**优点**: 理论上可以减少偏差
**缺点**: GPS估计不准时会放大误差

**策略2: 分层采样 (Stratified Sampling)**

**实现**:

```python
def stratified_batch_sampler(X, T, batch_size=128, n_strata=5):
    """
    确保每个batch中T的分布均匀
    """
    # 将T分层
    t_quantiles = np.quantile(T, np.linspace(0, 1, n_strata+1))
    strata = np.digitize(T, t_quantiles)
    
    # 从每层等量采样
    batch_indices = []
    per_stratum = batch_size // n_strata
    
    for s in range(1, n_strata+1):
        stratum_indices = np.where(strata == s)[0]
        sampled = np.random.choice(stratum_indices, per_stratum, replace=True)
        batch_indices.extend(sampled)
    
    return batch_indices
```

**优点**: 简单有效，确保模型见到各种T值
**缺点**: 如果某层样本极少，需要重复采样（可能过拟合）

**策略3: 数据增强 (Data Augmentation)**

**Mixup for T**:

```python
def mixup_treatment(X, T, Y, alpha=0.2):
    """
    在处理空间做插值，生成新样本
    """
    lam = np.random.beta(alpha, alpha)
    
    # 随机配对
    indices = np.random.permutation(len(X))
    
    X_mixed = lam * X + (1 - lam) * X[indices]
    T_mixed = lam * T + (1 - lam) * T[indices]
    Y_mixed = lam * Y + (1 - lam) * Y[indices]  # 假设线性混合合理
    
    return X_mixed, T_mixed, Y_mixed
```

**注意**: 只有当Y对T近似线性时才合理，否则可能引入bias

**策略4: 限制推断范围**

**最保守的方法**:

```python
def get_reliable_t_range(T, X, min_samples=10):
    """
    只在有足够数据的T区间做推断
    """
    t_bins = np.linspace(T.min(), T.max(), 20)
    
    # 统计每个bin的样本数
    hist, _ = np.histogram(T, bins=t_bins)
    
    # 找到充分支撑的区间
    valid_bins = hist >= min_samples
    
    t_min = t_bins[np.argmax(valid_bins)]
    t_max = t_bins[len(t_bins) - 1 - np.argmax(valid_bins[::-1])]
    
    return t_min, t_max

# 使用
t_min, t_max = get_reliable_t_range(T, X)
print(f"只在 [{t_min:.1f}, {t_max:.1f}] 区间做可靠推断")
```

**策略5: 主动学习 (Active Learning)**

如果可以收集新数据，优先采集稀疏区域:

```python
def select_samples_for_annotation(model, X_pool, T_grid, budget=100):
    """
    选择对哪些(X, T)组合做实验
    """
    # 计算不确定性（如果模型支持）
    uncertainty_scores = []
    
    for x in X_pool:
        # 评估在不同T下的不确定性
        uncertainties = []
        for t in T_grid:
            # 如果有多次采样
            samples = [model(x, t) for _ in range(10)]
            uncertainty = np.std(samples)
            uncertainties.append(uncertainty)
        
        # 选择最不确定的T
        max_uncertainty = np.max(uncertainties)
        uncertainty_scores.append(max_uncertainty)
    
    # 返回top-K不确定的样本
    topk = np.argsort(uncertainty_scores)[-budget:]
    
    return X_pool[topk], T_grid[...]  # 对应的最优T
```

**策略6: 多任务学习**

如果有相关任务（如预测点击率），联合训练可以缓解数据稀疏:

```python
class MultiTaskVCNet(nn.Module):
    def __init__(self, ...):
        self.shared_repr = ...  # 共享表示
        self.task1_head = ...   # 主任务: 预测消费
        self.task2_head = ...   # 辅助任务: 预测点击
    
    def forward(self, x, t):
        phi = self.shared_repr(x)
        
        # 两个任务共享表示，缓解数据稀疏
        y1 = self.task1_head(phi, t)
        y2 = self.task2_head(phi, t)
        
        return y1, y2
```

**实战组合策略**:

在优惠券场景的推荐做法:
1. **训练时**: 使用分层采样 + GPS加权
2. **评估时**: 分别报告不同T区间的性能
3. **推断时**: 对稀疏区域(t>30元)报告更宽的置信区间
4. **业务建议**: 对高价值用户在稀疏区域做小规模RCT验证

**监控指标**:

```python
def analyze_treatment_distribution(T, X):
    print("处理强度分布分析:")
    print(f"  均值: {T.mean():.2f}")
    print(f"  标准差: {T.std():.2f}")
    print(f"  偏度: {scipy.stats.skew(T):.2f}")  # >0表示右偏
    
    # 各quartile的样本数
    for q in [0.25, 0.5, 0.75]:
        t_q = np.quantile(T, q)
        print(f"  {int(q*100)}%分位数: {t_q:.1f}")
    
    # 稀疏性警告
    t_bins = np.linspace(T.min(), T.max(), 10)
    hist, _ = np.histogram(T, bins=t_bins)
    sparse_bins = np.where(hist < len(T) * 0.01)[0]  # <1%的bin
    
    if len(sparse_bins) > 0:
        print(f"  警告: {len(sparse_bins)}个区间样本稀疏(<1%)")
```

**结论**: 处理分布不均匀需要组合多种策略——训练时加权/采样，推断时限制范围，长期通过主动学习补充数据。

---

### 5. 在优惠券场景中，如何设计 A/B 测试来收集训练数据？

**参考答案**:

为连续处理（优惠券面额）设计A/B测试比二元处理复杂，需要在**探索-利用**之间权衡。

**挑战**:

1. **无限维的处理空间**: 面额可以是0-50元任意值
2. **成本约束**: 不能无限制地发券
3. **时效性**: 需要快速迭代，不能永远只探索

**策略1: 分层随机化 (Stratified Randomization)**

**设计**:

```
用户群体划分:
- 高价值用户 (LTV > 1000)
- 中价值用户 (500 < LTV ≤ 1000)
- 低价值用户 (LTV ≤ 500)

每个群体内随机分配:
群体1: 均匀分配 t ~ Uniform([20, 50])
群体2: 均匀分配 t ~ Uniform([10, 30])
群体3: 均匀分配 t ~ Uniform([0, 20])
```

**优点**:
- 确保每个X层在多个T值有数据
- 满足共同支撑假设

**实现**:

```python
def stratified_treatment_assignment(users, strata_rules):
    """
    分层随机分配处理强度
    
    Args:
        users: 用户特征DataFrame
        strata_rules: 每层的处理范围
    """
    assignments = []
    
    for user in users:
        # 确定用户所属层
        stratum = assign_stratum(user)
        
        # 从该层的处理范围随机采样
        t_min, t_max = strata_rules[stratum]
        treatment = np.random.uniform(t_min, t_max)
        
        assignments.append({
            'user_id': user.id,
            'features': user.features,
            'treatment': treatment,
            'stratum': stratum
        })
    
    return pd.DataFrame(assignments)
```

**策略2: 多臂老虎机 (Multi-Armed Bandit)**

对于连续处理，使用**连续臂老虎机**（Continuous-Armed Bandit）

**Gaussian Process Thompson Sampling**:

```python
class GPBandit:
    def __init__(self, t_range=(0, 50)):
        self.t_range = t_range
        self.gp = GaussianProcessRegressor()
        self.observations = []
    
    def select_treatment(self, user_features, explore_prob=0.2):
        """
        为用户选择优惠券面额
        """
        if len(self.observations) < 10 or np.random.rand() < explore_prob:
            # 探索: 随机采样
            return np.random.uniform(*self.t_range)
        else:
            # 利用: 基于GP的后验采样
            t_candidates = np.linspace(*self.t_range, 50)
            
            # 预测每个t的期望收益及不确定性
            mu, sigma = self.gp.predict(
                [[*user_features, t] for t in t_candidates],
                return_std=True
            )
            
            # Thompson Sampling: 从后验采样
            samples = mu + np.random.randn(len(t_candidates)) * sigma
            
            return t_candidates[np.argmax(samples)]
    
    def update(self, user_features, treatment, reward):
        """
        更新模型
        """
        self.observations.append([*user_features, treatment, reward])
        
        # 重新训练GP
        X = np.array([[*obs[:- 1]] for obs in self.observations])
        y = np.array([obs[-1] for obs in self.observations])
        self.gp.fit(X, y)
```

**优点**:
- 自动平衡探索-利用
- 数据高效

**缺点**:
- GP在高维特征下可能慢
- 需要在线更新

**策略3: 自适应实验设计 (Adaptive Design)**

**阶段式实验**:

```
阶段1 (Week 1-2): 宽泛探索
- 对所有用户，t ~ Uniform([0, 50])
- 收集基础数据

阶段2 (Week 3-4): 聚焦探索  
- 基于Week 1-2数据，识别"有希望"的区域
- 例如发现 t ∈ [15, 25] 最优
- 增加此区间的采样密度: 70%流量在[15,25]，30%在其他

阶段3 (Week 5+): 利用
- 基于模型预测，给每个用户最优面额
- 保留10%流量继续探索（防止模型过时）
```

**实现**:

```python
def adaptive_experiment_design(week, model, users):
    if week <= 2:
        # 阶段1: 均匀探索
        return np.random.uniform(0, 50, len(users))
    
    elif week <= 4:
        # 阶段2: 聚焦探索
        # 找到高回报区域
        promising_range = find_promising_range(model)
        
        treatments = []
        for user in users:
            if np.random.rand() < 0.7:
                # 70%在promising range
                t = np.random.uniform(*promising_range)
            else:
                # 30%全局探索
                t = np.random.uniform(0, 50)
            treatments.append(t)
        
        return np.array(treatments)
    
    else:
        # 阶段3: 主要利用
        treatments = []
        for user in users:
            if np.random.rand() < 0.1:
                # 10%探索
                t = np.random.uniform(0, 50)
            else:
                # 90%利用
                t = model.predict_optimal_treatment(user)
            treatments.append(t)
        
        return np.array(treatments)
```

**策略4: 贝叶斯优化 (Bayesian Optimization)**

将实验设计看作优化问题:

$$\max_{t} E[Y | X=x, T=t] \quad \text{s.t. 数据收集约束}$$

```python
from bayes_opt import BayesianOptimization

def design_next_batch(model, user_pool, batch_size=1000):
    """
    选择下一批实验的(user, treatment)组合
    """
    # 采集函数: Expected Improvement
    def acquisition(user_idx, t):
        user = user_pool[user_idx]
        
        # 当前最优值
        current_best = model.predict(user, model.current_best_t)
        
        # 预测均值和方差
        mu, sigma = model.predict_with_uncertainty(user, t)
        
        # EI = E[max(mu - current_best, 0)]
        improvement = mu - current_best
        z = improvement / (sigma + 1e-6)
        ei = improvement * norm.cdf(z) + sigma * norm.pdf(z)
        
        return ei
    
    # 优化采集函数
    selected_pairs = []
    for _ in range(batch_size):
        best_ei = -np.inf
        best_pair = None
        
        for user_idx in range(len(user_pool)):
            for t in np.linspace(0, 50, 20):
                ei = acquisition(user_idx, t)
                if ei > best_ei:
                    best_ei = ei
                    best_pair = (user_idx, t)
        
        selected_pairs.append(best_pair)
    
    return selected_pairs
```

**策略5: 分桶近似 (Discretization)**

如果连续优化太复杂，可以离散化:

```
将面额离散为5档:
- 0元 (控制组)
- 10元
- 20元
- 30元
- 50元

然后做标准的5组A/B测试
```

**权衡**:
- ✅ 简化设计和分析
- ❌ 无法找到10-20元之间的最优点

**最佳实践: 混合策略**

```python
class HybridExperimentDesign:
    """
    混合策略: 分层 + 自适应 + 离散化
    """
    def __init__(self):
        # 离散化为几个关键档位
        self.key_amounts = [0, 5, 10, 15, 20, 30, 50]
        
        # 每个档位的探索概率（初始均匀，后续自适应）
        self.probs = np.ones(len(self.key_amounts)) / len(self.key_amounts)
    
    def assign_treatment(self, user, week):
        # 确定用户层
        stratum = self.get_stratum(user)
        
        # 过滤适合该层的档位
        if stratum == 'high_value':
            valid_amounts = [t for t in self.key_amounts if t >= 15]
        elif stratum == 'low_value':
            valid_amounts = [t for t in self.key_amounts if t <= 20]
        else:
            valid_amounts = self.key_amounts
        
        # 基于当前概率采样
        # (概率会根据观察到的收益自适应更新)
        return np.random.choice(valid_amounts, p=self.probs_normalized(valid_amounts))
    
    def update_probs(self, observations):
        """
        基于观察数据更新探索概率
        """
        # 计算每个档位的平均回报
        rewards_by_amount = {}
        for amount in self.key_amounts:
            mask = observations['treatment'] == amount
            if mask.sum() > 0:
                rewards_by_amount[amount] = observations.loc[mask, 'reward'].mean()
        
        # Softmax更新（温度参数控制探索-利用权衡）
        temperature = max(0.1, 1.0 - week * 0.1)  # 逐渐降低温度
        new_probs = softmax([rewards_by_amount.get(a, 0) / temperature 
                             for a in self.key_amounts])
        
        self.probs = new_probs
```

**监控指标**:

```python
def experiment_health_check(data):
    """
    检查实验数据质量
    """
    print("="*60)
    print("实验数据健康度检查")
    print("="*60)
    
    # 1. 处理分布
    print("\n1. 处理分布:")
    print(data['treatment'].describe())
    
    # 2. 各区间样本量
    print("\n2. 各面额区间样本量:")
    for t_min, t_max in [(0,10), (10,20), (20,30), (30,50)]:
        mask = (data['treatment'] >= t_min) & (data['treatment'] < t_max)
        print(f"  [{t_min}-{t_max}元): {mask.sum()} ({mask.mean():.1%})")
    
    # 3. 各层的覆盖度
    print("\n3. 用户层的处理覆盖:")
    for stratum in data['stratum'].unique():
        stratum_data = data[data['stratum'] == stratum]
        print(f"  {stratum}:")
        print(f"    样本数: {len(stratum_data)}")
        print(f"    处理范围: [{stratum_data['treatment'].min():.1f}, "
              f"{stratum_data['treatment'].max():.1f}]")
    
    # 4. 时间趋势
    data['week'] = pd.to_datetime(data['timestamp']).dt.isocalendar().week
    print("\n4. 每周数据量:")
    print(data.groupby('week').size())
```

**结论**: 
- **短期**(Week 1-2): 分层随机化，确保共同支撑
- **中期**(Week 3-6): 自适应设计，聚焦高价值区域
- **长期**: 利用为主(90%)，保留探索(10%)
- 始终监控数据质量，避免某些区域样本过少

---

### 6. 如果有预算约束，如何在整体上优化优惠券分配？

**参考答案**:

这是一个**带约束的因果优化问题**，目标是在预算约束下最大化总收益。

**问题形式化**:

$$\begin{align}
\max_{t_1, ..., t_N} \quad & \sum_{i=1}^{N} [E[Y_i | X_i, T_i=t_i] - c \cdot t_i] \\
\text{s.t.} \quad & \sum_{i=1}^{N} t_i \leq B \\
& 0 \leq t_i \leq t_{max}, \quad \forall i
\end{align}$$

其中:
- $N$: 用户数
- $t_i$: 给用户i的券面额
- $E[Y_i | X_i, T_i=t_i]$: 预期消费（由VCNet预测）
- $c$: 成本系数（通常=1）
- $B$: 总预算

**策略1: 贪心分配 (Greedy Allocation)**

**核心思想**: 优先给边际收益最大的用户发券

```python
def greedy_allocation(model, users, budget):
    """
    贪心分配优惠券
    """
    N = len(users)
    allocations = np.zeros(N)
    remaining_budget = budget
    
    # 对每个用户，找到最优面额
    optimal_amounts = []
    marginal_benefits = []
    
    for i, user in enumerate(users):
        # 在[0, remaining_budget]范围内搜索最优t
        t_values = np.linspace(0, min(50, remaining_budget), 100)
        
        # 预测净收益
        net_benefits = []
        for t in t_values:
            y_pred = model.predict(user, t)
            net_benefit = y_pred - t  # 收益 - 成本
            net_benefits.append(net_benefit)
        
        optimal_idx = np.argmax(net_benefits)
        optimal_t = t_values[optimal_idx]
        optimal_benefit = net_benefits[optimal_idx]
        
        optimal_amounts.append(optimal_t)
        marginal_benefits.append(optimal_benefit)
    
    # 按边际收益排序
    sorted_indices = np.argsort(marginal_benefits)[::-1]
    
    # 贪心分配
    for idx in sorted_indices:
        if remaining_budget <= 0:
            break
        
        t = min(optimal_amounts[idx], remaining_budget)
        allocations[idx] = t
        remaining_budget -= t
    
    return allocations
```

**复杂度**: O(N log N)
**最优性**: 近似最优（无理论保证）

**策略2: 线性规划 (Linear Programming)**

如果剂量-响应曲线是**线性或分段线性**的，可以用LP求精确解。

**分段线性近似**:

```python
from scipy.optimize import linprog

def lp_allocation(model, users, budget, n_segments=5):
    """
    将非线性剂量-响应曲线分段线性化，然后用LP求解
    """
    N = len(users)
    
    # 对每个用户，将[0, 50]分成n_segments段
    t_breakpoints = np.linspace(0, 50, n_segments + 1)
    
    # 变量: t_ij 表示用户i在第j段的分配量
    # 总变量数 = N * n_segments
    
    # 目标函数系数
    c = []
    for i, user in enumerate(users):
        for j in range(n_segments):
            t_start = t_breakpoints[j]
            t_end = t_breakpoints[j+1]
            t_mid = (t_start + t_end) / 2
            
            # 预测该段的平均收益
            y_pred = model.predict(user, t_mid)
            benefit_per_unit = (y_pred - t_mid) / (t_end - t_start)
            
            c.append(-benefit_per_unit)  # 负号因为linprog是最小化
    
    # 约束1: 预算约束
    A_budget = np.zeros((1, N * n_segments))
    for i in range(N):
        for j in range(n_segments):
            A_budget[0, i * n_segments + j] = t_breakpoints[j+1] - t_breakpoints[j]
    b_budget = [budget]
    
    # 约束2: 每个用户的段分配量之和 ≤ 1 (归一化)
    A_user = np.zeros((N, N * n_segments))
    for i in range(N):
        for j in range(n_segments):
            A_user[i, i * n_segments + j] = 1
    b_user = [1] * N
    
    # 变量界: 0 ≤ t_ij ≤ 1
    bounds = [(0, 1)] * (N * n_segments)
    
    # 求解
    result = linprog(c, A_ub=np.vstack([A_budget, A_user]),
                     b_ub=b_budget + b_user, bounds=bounds, method='highs')
    
    # 恢复实际分配
    allocations = np.zeros(N)
    for i in range(N):
        for j in range(n_segments):
            segment_allocation = result.x[i * n_segments + j]
            allocations[i] += segment_allocation * (t_breakpoints[j+1] - t_breakpoints[j])
    
    return allocations
```

**优点**: 精确最优（在分段线性近似下）
**缺点**: 计算复杂，仅适合中小规模

**策略3: 拉格朗日松弛 (Lagrangian Relaxation)**

对于大规模问题，用拉格朗日乘子法:

$$L(t_1, ..., t_N, \lambda) = \sum_{i=1}^{N} [E[Y_i|X_i, t_i] - t_i] - \lambda \left(\sum_{i=1}^{N} t_i - B\right)$$

**算法**:

```python
def lagrangian_allocation(model, users, budget, max_iter=100):
    """
    拉格朗日松弛法求解
    """
    N = len(users)
    lambda_val = 0.1  # 初始拉格朗日乘子
    
    for iteration in range(max_iter):
        allocations = np.zeros(N)
        
        # 对每个用户，独立优化
        for i, user in enumerate(users):
            # 最优化 E[Y_i|X_i, t_i] - t_i - lambda * t_i
            #       = E[Y_i|X_i, t_i] - (1 + lambda) * t_i
            
            t_values = np.linspace(0, 50, 100)
            objectives = []
            
            for t in t_values:
                y_pred = model.predict(user, t)
                obj = y_pred - (1 + lambda_val) * t
                objectives.append(obj)
            
            optimal_t = t_values[np.argmax(objectives)]
            allocations[i] = optimal_t
        
        # 检查约束
        total_budget_used = allocations.sum()
        
        if abs(total_budget_used - budget) < 1:
            break
        
        # 更新拉格朗日乘子
        if total_budget_used > budget:
            lambda_val += 0.01  # 增大lambda，减少分配
        else:
            lambda_val -= 0.01  # 减小lambda，增加分配
        
        lambda_val = max(0, lambda_val)  # 非负约束
    
    # 最终调整到精确预算
    if allocations.sum() > budget:
        # 按比例缩减
        allocations *= budget / allocations.sum()
    elif allocations.sum() < budget:
        # 贪心补充
        remaining = budget - allocations.sum()
        # 给边际收益最大的用户补充
        # ...
    
    return allocations
```

**策略4: 排名优化 (Ranking Optimization)**

实践中常用的简化方法:

```python
def ranking_allocation(model, users, budget):
    """
    基于用户价值排名分配
    """
    N = len(users)
    
    # 计算每个用户的"uplift per dollar"
    uplift_efficiency = []
    
    for user in users:
        # 比较发20元券 vs 不发
        y_treat = model.predict(user, t=20/50)  # 归一化
        y_control = model.predict(user, t=0)
        
        uplift = y_treat - y_control
        efficiency = uplift / 20  # 每元券带来的uplift
        
        uplift_efficiency.append(efficiency)
    
    # 排名
    ranked_indices = np.argsort(uplift_efficiency)[::-1]
    
    # 分配
    allocations = np.zeros(N)
    remaining_budget = budget
    
    for idx in ranked_indices:
        if remaining_budget <= 0:
            break
        
        # 给该用户发最优面额（在剩余预算内）
        optimal_t = find_optimal_treatment(model, users[idx], 
                                            max_t=min(50, remaining_budget))
        
        allocations[idx] = optimal_t
        remaining_budget -= optimal_t
    
    return allocations
```

**策略5: 强化学习 (Reinforcement Learning)**

将预算分配看作序贯决策问题:

```python
class BudgetAllocationMDP:
    """
    将预算分配建模为MDP
    
    State: (当前用户特征, 剩余预算, 已服务用户数)
    Action: 给当前用户的券面额
    Reward: 净收益
    """
    def __init__(self, model, users, total_budget):
        self.model = model
        self.users = users
        self.total_budget = total_budget
        
        self.current_user_idx = 0
        self.remaining_budget = total_budget
    
    def get_state(self):
        if self.current_user_idx >= len(self.users):
            return None  # 终止状态
        
        user_features = self.users[self.current_user_idx]
        return {
            'user_features': user_features,
            'remaining_budget': self.remaining_budget,
            'users_served': self.current_user_idx,
            'total_users': len(self.users)
        }
    
    def step(self, action_t):
        """
        执行动作: 给当前用户分配面额t
        """
        user = self.users[self.current_user_idx]
        
        # 计算奖励
        y_pred = self.model.predict(user, action_t)
        reward = y_pred - action_t
        
        # 更新状态
        self.remaining_budget -= action_t
        self.current_user_idx += 1
        
        next_state = self.get_state()
        done = (next_state is None)
        
        return next_state, reward, done
```

然后用DQN/PPO等RL算法训练策略。

**优点**: 能学习复杂的序贯决策策略
**缺点**: 训练复杂，需要大量模拟

**实战推荐: 两阶段方法**

```python
class TwoStageAllocation:
    """
    第一阶段: 用VCNet预测每个用户的最优面额
    第二阶段: 在预算约束下优化总体分配
    """
    def __init__(self, vcnet_model):
        self.model = vcnet_model
    
    def allocate(self, users, budget):
        N = len(users)
        
        # 阶段1: 计算无约束最优
        unconstrained_optimal = np.zeros(N)
        expected_benefits = np.zeros(N)
        
        for i, user in enumerate(users):
            # 找到ROI最大的面额
            result = find_optimal_dosage(self.model, user, cost_per_unit=1.0)
            unconstrained_optimal[i] = result['optimal_amount']
            expected_benefits[i] = result['net_benefit']
        
        total_needed = unconstrained_optimal.sum()
        
        if total_needed <= budget:
            # 预算充足，直接分配
            return unconstrained_optimal
        else:
            # 预算不足，需要优化
            # 使用贪心 + 边际分析
            return self._constrained_optimization(
                users, budget, unconstrained_optimal, expected_benefits
            )
    
    def _constrained_optimization(self, users, budget, optimal_t, benefits):
        """
        预算约束下的优化
        """
        N = len(users)
        allocations = np.zeros(N)
        
        # 按边际收益排序
        marginal_roi = benefits / (optimal_t + 1e-6)  # ROI
        sorted_indices = np.argsort(marginal_roi)[::-1]
        
        remaining_budget = budget
        
        for idx in sorted_indices:
            if remaining_budget <= 0:
                break
            
            # 分配该用户的最优面额（不超过剩余预算）
            t = min(optimal_t[idx], remaining_budget)
            allocations[idx] = t
            remaining_budget -= t
        
        return allocations
```

**监控与调整**:

```python
def evaluate_allocation_strategy(allocations, users, model, actual_outcomes=None):
    """
    评估分配策略
    """
    # 预测指标
    predicted_revenue = 0
    total_cost = 0
    
    for i, (user, t) in enumerate(zip(users, allocations)):
        y_pred = model.predict(user, t / 50)  # 归一化
        predicted_revenue += y_pred
        total_cost += t
    
    predicted_roi = (predicted_revenue - total_cost) / total_cost
    
    print(f"预测总收入: {predicted_revenue:.2f}")
    print(f"总成本: {total_cost:.2f}")
    print(f"预测 ROI: {predicted_roi:.2%}")
    
    # 如果有实际结果，计算真实ROI
    if actual_outcomes is not None:
        actual_revenue = actual_outcomes.sum()
        actual_roi = (actual_revenue - total_cost) / total_cost
        
        print(f"\n实际总收入: {actual_revenue:.2f}")
        print(f"实际 ROI: {actual_roi:.2%}")
        print(f"预测偏差: {(predicted_revenue - actual_revenue) / actual_revenue:.2%}")
```

**与 Part 6 的联系**:

这个问题与Part 6的预算分配优化密切相关:
- Part 5 (VCNet): 学习个体剂量-响应函数 μ(t, x)
- Part 6: 在总体上优化预算分配策略

结合使用:
1. 用VCNet估计每个用户的最优处理
2. 用Part 6的方法在预算约束下优化总体分配
3. 迭代更新: 新数据→重训练VCNet→重新分配

**结论**: 
预算约束下的优惠券分配是一个带约束的因果优化问题。实战中推荐:
- **小规模** (<1000用户): LP精确求解
- **中规模** (1000-10万): 贪心算法
- **大规模** (>10万): 分层+贪心+并行计算
- 始终监控实际ROI，迭代优化

---

*这些详细的答案展示了VCNet在真实业务中的应用深度！* 🎓

## 总结

| 概念 | 定义 | 重要性 |
|------|------|--------|
| **连续处理** | T 取连续值而非 0/1 | 更真实的业务场景 |
| **剂量-响应曲线** | Y(t) 关于 t 的函数 | 核心估计目标 |
| **变系数网络** | W(t) · φ(X) | VCNet 核心创新 |
| **样条基函数** | 保证曲线光滑 | 正则化作用 |
| **最优剂量** | argmax_t μ(t, x) | 业务决策依据 |

### 与其他方法的对比

| 方法 | 处理类型 | 输出 | 适用场景 |
|------|---------|------|----------|
| TARNet/DragonNet | 二元 | μ₀(x), μ₁(x) | 发不发 |
| GPS + 回归 | 连续 | μ(t, x) | 简单关系 |
| VCNet | 连续 | μ(t, x) | 复杂非线性 |
| DRNet | 连续 | μ(t, x) | 多剂量区间 |

### 何时使用 VCNet？

- ✅ 处理强度是连续的（面额、价格、时长）
- ✅ 需要估计完整的剂量-响应曲线
- ✅ 剂量-响应关系复杂非线性
- ❌ 处理只有少数几个档位（用多处理方法）
- ❌ 数据量很小

---

## 延伸阅读

- **原论文**: Nie, L., Ye, M., Liu, Q., & Nicolae, D. (2021). VCNet and Functional Targeted Regularization For Learning Causal Effects of Continuous Treatments. ICLR 2021.
- **相关工作**: DRNet, GPS, SCIGAN

---

*恭喜你完成了 VCNet 的学习！* 🎉